# RusTorch CoreML 통합 - Python 바인딩

이 노트북은 Python 바인딩을 통해 RusTorch의 CoreML 기능을 사용하는 방법을 보여줍니다.

## 설정 및 임포트

In [ ]:
# RusTorch Python 바인딩 임포트
try:
    import rustorch
    print(f"✅ RusTorch 버전: {rustorch.__version__}")
    print(f"📝 설명: {rustorch.__description__}")
    print(f"👥 작성자: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ RusTorch 임포트 실패: {e}")
    print("maturin develop로 빌드해주세요")
    exit(1)

import numpy as np
import platform

print(f"🖥️ 플랫폼: {platform.system()} {platform.release()}")
print(f"🐍 Python 버전: {platform.python_version()}")

## CoreML 가용성 확인

In [ ]:
# CoreML 기능 확인
try:
    # CoreML 사용 가능 여부 확인
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML 사용 가능: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML을 사용할 수 있습니다!")
        
        # 디바이스 정보 가져오기
        device_info = rustorch.get_coreml_device_info()
        print("📱 CoreML 디바이스 정보:")
        print(device_info)
    else:
        print("⚠️ CoreML을 사용할 수 없습니다")
        if platform.system() != "Darwin":
            print("CoreML은 macOS에서만 사용 가능합니다")
        else:
            print("CoreML 기능이 활성화되지 않았을 수 있습니다")
            
except AttributeError:
    print("❌ CoreML 함수를 찾을 수 없습니다")
    print("CoreML 기능으로 빌드되지 않았을 수 있습니다")
    coreml_available = False
except Exception as e:
    print(f"❌ CoreML 확인 중 오류: {e}")
    coreml_available = False

## CoreML 디바이스 생성 및 작업

In [ ]:
if coreml_available:
    try:
        # CoreML 디바이스 생성
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ CoreML 디바이스 생성됨: {device}")
        
        # 디바이스 정보 가져오기
        print(f"🆔 디바이스 ID: {device.device_id()}")
        print(f"✅ 사용 가능: {device.is_available()}")
        print(f"💾 메모리 제한: {device.memory_limit()} 바이트")
        print(f"🧮 연산 유닛 제한: {device.compute_units_limit()}")
        print(f"📚 모델 캐시 크기: {device.model_cache_size()}")
        
        # 캐시 정리
        device.cleanup_cache()
        print("🧹 캐시 정리됨")
        
    except Exception as e:
        print(f"❌ CoreML 디바이스 작업 오류: {e}")
else:
    print("⚠️ CoreML을 사용할 수 없어 디바이스 작업을 건너뜁니다")

## CoreML 백엔드 구성

In [ ]:
if coreml_available:
    try:
        # CoreML 백엔드 구성 생성
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ 백엔드 구성: {config}")
        
        # 구성 값 확인 및 수정
        print(f"📊 캐싱 활성화: {config.enable_caching}")
        print(f"🗂️ 최대 캐시 크기: {config.max_cache_size}")
        print(f"📈 프로파일링 활성화: {config.enable_profiling}")
        print(f"🔄 자동 폴백: {config.auto_fallback}")
        
        # 구성 수정
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 업데이트된 구성: {config}")
        
        # CoreML 백엔드 생성
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 CoreML 백엔드: {backend}")
        print(f"✅ 백엔드 사용 가능: {backend.is_available()}")
        
        # 백엔드 통계 가져오기
        stats = backend.get_stats()
        print(f"📊 백엔드 통계: {stats}")
        print(f"   총 작업 수: {stats.total_operations}")
        print(f"   캐시 히트: {stats.cache_hits}")
        print(f"   캐시 미스: {stats.cache_misses}")
        print(f"   폴백 작업: {stats.fallback_operations}")
        print(f"   캐시 히트율: {stats.cache_hit_rate():.2%}")
        print(f"   폴백율: {stats.fallback_rate():.2%}")
        print(f"   평균 실행 시간: {stats.average_execution_time_ms:.2f}ms")
        
        # 캐시 정리
        backend.cleanup_cache()
        print("\n🧹 백엔드 캐시 정리됨")
        
    except Exception as e:
        print(f"❌ CoreML 백엔드 작업 오류: {e}")
else:
    print("⚠️ CoreML을 사용할 수 없어 백엔드 작업을 건너뜁니다")

## 기본 텐서 연산 (CPU)

CoreML과 비교하기 위해 먼저 CPU에서 기본 연산을 수행합니다.

In [ ]:
try:
    # 기본 텐서 생성 및 연산
    print("🧮 기본 텐서 연산 (CPU)")
    
    # NumPy 배열에서 텐서 생성 (단순화된 인터페이스)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 행렬 A 형태: {data_a.shape}")
    print(f"📐 행렬 B 형태: {data_b.shape}")
    
    # NumPy로 행렬 곱셈 (비교용)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ NumPy matmul 결과 형태: {numpy_result.shape}")
    print(f"📊 결과 (첫 번째 요소들): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 CPU 연산 완료")
    
except Exception as e:
    print(f"❌ 텐서 연산 오류: {e}")

## 성능 비교 시뮬레이션

In [ ]:
import time

def benchmark_matrix_operations():
    """다른 행렬 크기로 성능 비교"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 성능 비교:")
    print("크기\t\tCPU 시간 (ms)\t예상 CoreML (ms)")
    print("-" * 50)
    
    for size in sizes:
        # CPU 실행 시간 측정
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # 예상 CoreML 시간 (가상)
        # 실제 구현에서는 CoreML 백엔드의 실제 측정값 사용
        expected_coreml_time = cpu_time * 0.6  # 가정: CoreML이 40% 빠름
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 참고: CoreML 시간은 가상입니다. 실제 값은 구체적인 구현에 따라 달라집니다.")

## 디바이스 선택 시뮬레이션

In [ ]:
def simulate_device_selection():
    """스마트 디바이스 선택 시뮬레이션"""
    
    operations = [
        ("작은 행렬 곱셈", (16, 16), "CPU"),
        ("중간 행렬 곱셈", (128, 128), "Metal GPU"),
        ("큰 행렬 곱셈", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("활성화 함수", (32, 64, 128, 128), "Metal GPU"),
        ("작은 컨볼루션", (1, 3, 32, 32), "CPU"),
        ("큰 컨볼루션", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("복소수 연산", (128, 128), "Metal GPU"),  # CoreML 미지원
        ("통계 분포", (1000,), "CPU"),  # CoreML 미지원
    ]
    
    print("🎯 스마트 디바이스 선택 시뮬레이션:")
    print("연산\t\t\t텐서 형태\t\t선택된 디바이스")
    print("-" * 70)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<23}\t{shape_str:<15}\t{device}")
    
    print("\n📝 선택 논리:")
    print("  • 작은 연산: CPU (오버헤드 방지)")
    print("  • 중간 연산: Metal GPU (균형)")
    print("  • 큰 연산: CoreML (최적화)")
    print("  • 지원되지 않는 연산: GPU/CPU 폴백")

simulate_device_selection()

## 실용적인 예제: 간단한 신경망 레이어

In [ ]:
def simulate_neural_network_layer():
    """신경망 레이어 시뮬레이션"""
    
    print("🧠 신경망 레이어 시뮬레이션:")
    
    # 배치 크기 및 레이어 구성
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10개 클래스
    
    print(f"📊 배치 크기: {batch_size}")
    print(f"🔢 입력 특성: {input_features}")
    print(f"🧮 은닉 특성: {hidden_features}")
    print(f"🎯 출력 특성: {output_features}")
    
    # 순전파 시뮬레이션
    steps = [
        ("입력 → 은닉", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("ReLU 활성화", f"({batch_size}, {hidden_features})", "Metal"),
        ("은닉 → 출력", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 순전파 시뮬레이션:")
    total_time = 0
    
    for step, shape, device in steps:
        # 가상 실행 시간 (ms)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ 총 순전파 시간: {total_time:.2f}ms")
    print(f"🚀 예상 처리량: {1000/total_time:.0f} 배치/초")

simulate_neural_network_layer()

## 요약 및 다음 단계

In [ ]:
print("📋 RusTorch CoreML 통합 요약:")
print()
print("✅ 완료된 항목:")
print("  • Jupyter 환경 설정")
print("  • Rust 커널 및 Python 바인딩 생성")
print("  • CoreML 가용성 확인")
print("  • 디바이스 관리 및 구성")
print("  • 백엔드 통계 및 프로파일링")
print("  • 스마트 디바이스 선택")
print()
print("🚧 향후 개발:")
print("  • 실제 CoreML 연산 구현")
print("  • 성능 벤치마킹")
print("  • 더 많은 활성화 함수 및 레이어 타입")
print("  • 오류 처리 개선")
print("  • 메모리 최적화")
print()
print("🎯 권장 다음 단계:")
print("  1. 실제 CoreML 모델 로드 및 테스트")
print("  2. Metal과 CoreML 성능 비교")
print("  3. 실제 딥러닝 워크플로우로 테스트")
print("  4. 프로덕션 환경에서 평가")

if coreml_available:
    print("\n🎉 축하합니다! CoreML을 사용할 수 있으며 모든 기능을 테스트할 수 있습니다.")
else:
    print("\n⚠️ CoreML을 사용할 수 없지만 기본 기능은 작동합니다.")
    print("   macOS에서 CoreML 기능을 활성화하여 빌드하는 것을 권장합니다.")